In [5]:
from ml_fertilizers.utils import load_data

# DATA LOAD

In [6]:
from IPython.display import display
train, test = load_data()
# train = train.set_index("id")
# test = test.set_index("id")
display(train)

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP
...,...,...,...,...,...,...,...,...,...,...
749995,749995,25,69,30,Clayey,Maize,8,16,6,28-28
749996,749996,37,64,58,Loamy,Sugarcane,38,8,20,17-17-17
749997,749997,35,68,59,Sandy,Ground Nuts,6,11,29,10-26-26
749998,749998,31,68,29,Red,Cotton,9,11,12,20-20


# FEATURE ENGINEERING

In [15]:
import pickle as pkl
import numpy as np
from typing import Any, Dict, List, Optional, Tuple, Union, cast
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from autofeat import AutoFeatClassifier
 

def engineer_features(X: pd.DataFrame, autofeat_cls: Union[bool, Optional[AutoFeatClassifier]] = False) -> Tuple[pd.DataFrame, Dict[str, List[str]], Union[AutoFeatClassifier, bool]]:
    raw_num_features = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Phosphorous', 'Potassium']
    raw_cat_features = ['Crop', 'Soil']

    X = X.copy()
    X = X.rename(columns={
        "Soil Type": "Soil",
        "Crop Type": "Crop",
    })


    # X['Crop_x_Soil'] = X['Crop'] + '_' + X['Soil']
    X['Env_Stress_Index'] = X['Temparature'] *0.4 + X['Humidity'] * 0.3 + X['Moisture'] * 0.3
    X['NPK_Index'] = X['Nitrogen'] * 0.5 + X['Phosphorous'] * 0.3 + X['Potassium'] * 0.2
    X['Temp_bin'] = pd.cut(X['Temparature'], bins=[-float('inf'), 15, 25, 35, float('inf')], labels=['low', 'medium', 'high', 'very_high'])
    X['Humidity_bin'] = pd.cut(X['Humidity'], bins=[-float('inf'), 30, 50, 70, float('inf')], labels=['low', 'medium', 'high', 'very_high'])
    X['Moisture_bin'] = pd.cut(X['Moisture'], bins=[-float('inf'), 20, 40, 60, float('inf')], labels=['low', 'medium', 'high', 'very_high'])
    X['PCA_Temparature'] = PCA(n_components=2).fit_transform(X[['Temparature', 'Humidity', 'Moisture']])[:, 0]



    print("Autofeating features...")

    if isinstance(autofeat_cls, bool):
        print("Skipping autofeat feature engineering.")
        X_autofeat = pd.DataFrame()
    elif autofeat_cls is None:
        autofeat_cls = AutoFeatClassifier(verbose=0, n_jobs=-1, feateng_steps=2, categorical_cols=raw_cat_features)
        X_autofeat = cast(pd.DataFrame, autofeat_cls.fit_transform(X[raw_num_features + raw_cat_features], X['Fertilizer Name'])) # type: ignore
        print("Autofeat columns:", X_autofeat.columns.tolist())
    else:
        X_autofeat = cast(pd.DataFrame, autofeat_cls.transform(X[raw_num_features + raw_cat_features]))
        print("Autofeat columns:", X_autofeat.columns.tolist())
    
    
    

    X_final = pd.concat([X, X_autofeat], axis=1)
    X_final = X_final.loc[:, ~X_final.columns.duplicated()]


    final_dict = {
        "num_features": X_final.select_dtypes(include=['number']).columns.tolist(),
        "cat_features": X_final.drop(columns=['Fertilizer Name']).select_dtypes(include=['object', 'category']).columns.tolist(),
        "autofeat_features": X_autofeat.columns.tolist() if not isinstance(autofeat_cls, bool) else [],
    }
    return X_final, final_dict, autofeat_cls



def engineer_simple(X: pd.DataFrame, features_in: Optional[List[str]]) -> pd.DataFrame:
    features_in = features_in or []
    X = X.set_index("id")
    X = X.rename(
        columns={
            "Soil Type": "Soil",
            "Crop Type": "Crop",
        }
    )
    # NOTE: ENGINERRING
    # fmt: off
    if 'Env_Stress_Index' in features_in:
        X["Env_Stress_Index"] = X["Temparature"] * 0.4 + X["Humidity"] * 0.3 + X["Moisture"] * 0.3
    if 'NPK_Index' in features_in:
        X["NPK_Index"] = X["Nitrogen"] * 0.5 + X["Phosphorous"] * 0.3 + X["Potassium"] * 0.2
    if any(pd.Series(features_in).str.contains(r'^Temp_bin\.*$')):
        X["Temp_bin"] = pd.Categorical.from_codes(np.digitize(X["Temparature"].values, bins=[-np.inf, 15, 30, 45, np.inf]) - 1, categories=["low", "medium", "high", "very_high"])  # type: ignore
    if any(pd.Series(features_in).str.contains(r'^Humidity_bin\.*$')):
        X['Humidity_bin'] = pd.Categorical.from_codes(np.digitize(X["Humidity"].values, bins=[-np.inf, 30, 50, 70, np.inf]) - 1, categories=["low", "medium", "high", "very_high"])  # type: ignore
    if any(pd.Series(features_in).str.contains(r'^Moisture_bin\.*$')):
        X['Moisture_bin'] = pd.Categorical.from_codes(np.digitize(X["Moisture"].values, bins=[-np.inf, 20, 40, 60, np.inf]) - 1, categories=["low", "medium", "high", "very_high"])  # type: ignore

    if 'SoilNutrients' in features_in:
        X['SoilNutrients'] = X['Nitrogen'] + X['Phosphorous'] + X['Potassium']
    if 'SoilNutrient_Ratio' in features_in:
        X["SoilNutrient_Ratio"] = X["Nitrogen"] / (X["Potassium"] + X["Phosphorous"] + 1)
    if 'Temp_Humidity' in features_in:
        X["Temp_Humidity"] = X["Temparature"] * X["Humidity"]
    if 'Temp_Moisture' in features_in:
        X["Temp_Moisture"] = X["Temparature"] * X["Moisture"]
    # fmt: on
    cat_features = ["Crop", "Soil", "Temp_bin", "Humidity_bin", "Moisture_bin"]
    for col in cat_features:
        if col in X.columns:
            X[col] = X[col].astype("category")

    X_cat = pd.DataFrame()
    if any(pd.Series(features_in).str.contains(r"^Soil_[A-Za-z]+$")):
        X_cat = pd.get_dummies(
            X["Soil"], prefix="Soil", drop_first=False, sparse=True, dtype="uint8"
        )
    if any(pd.Series(features_in).str.contains(r"^Crop_[A-Za-z]+$")):
        X_cat = pd.concat(
            [
                X_cat,
                pd.get_dummies(
                    X["Crop"],
                    prefix="Crop",
                    drop_first=False,
                    sparse=True,
                    dtype="uint8",
                ),
            ],
            axis=1,
        )
    if any(pd.Series(features_in).str.contains(r"^Temp_bin_[A-Za-z]+$")):
        X_cat = pd.concat(
            [
                X_cat,
                pd.get_dummies(
                    X["Temp_bin"],
                    prefix="Temp_bin",
                    drop_first=False,
                    sparse=True,
                    dtype="uint8",
                ),
            ],
            axis=1,
        )
    if any(pd.Series(features_in).str.contains(r"^Humidity_bin_[A-Za-z]+$")):
        X_cat = pd.concat(
            [
                X_cat,
                pd.get_dummies(
                    X["Humidity_bin"],
                    prefix="Humidity_bin",
                    drop_first=False,
                    sparse=True,
                    dtype="uint8",
                ),
            ],
            axis=1,
        )
    if any(pd.Series(features_in).str.contains(r"^Moisture_bin_[A-Za-z]+$")):
        X_cat = pd.concat(
            [
                X_cat,
                pd.get_dummies(
                    X["Moisture_bin"],
                    prefix="Moisture_bin",
                    drop_first=False,
                    sparse=True,
                    dtype="uint8",
                ),
            ],
            axis=1,
        )

    if not X_cat.empty:
        X_final = pd.concat([X, X_cat], axis=1)
    else:
        X_final = X.copy()

    X = X_cat = None  # type: ignore
    del X_cat, X

    # Change int64 to uint16
    for col in X_final.select_dtypes(include=["int64"]).columns:
        X_final[col] = X_final[col].astype("uint16")
            
    return X_final


def create_preprocessor(features_in: Optional[List[str]] = None) -> Pipeline:

    pipeline_steps: List[Tuple[str, Any]] = [
        (
            "simple_engineering",
            FunctionTransformer( 
                lambda df: engineer_simple(df, features_in=features_in)
            ),
        )
    ]

    if features_in is not None and any(
        pd.Series(features_in).str.contains(r"^pca\d+$")
    ):

        ct_step = (
            (
                "ct",
                ColumnTransformer(
                    transformers=(
                        [
                            (
                                "temp_pca",
                                PCA(n_components=2),
                                ["Temparature", "Humidity", "Moisture"],
                            ),
                            (
                                "temp_stuff",
                                "passthrough",
                                ["Temparature", "Humidity", "Moisture"],
                            ),
                        ]
                    ),
                    remainder="passthrough",
                ),
            )
        )
        pipeline_steps.append(ct_step) # type: ignore

    pipeline_steps.append(
        (
            "remove_prefixes",
            FunctionTransformer(
                lambda df: df.rename(
                    columns=lambda x: x.split("__")[-1] if "__" in x else x
                )
            ),
        )
    )

    preprocessor = Pipeline(steps=pipeline_steps).set_output(transform="pandas")
    return preprocessor  # type: ignore


# eng_train, feat_dict, auto_cls = engineer_features(train, autofeat_cls=False)
preprocessor = create_preprocessor(features_in=['Env_Stress_Index',
        'NPK_Index',
        'Temp_bin',
        'Humidity_bin',
        'Moisture_bin',
        'SoilNutrients',
        'SoilNutrient_Ratio',
        'Temp_Humidity',
        'Temp_Moisture',
        'Soil',
        'Crop',
        'pca0'])
eng_train = preprocessor.fit_transform(train)
ohe_features = [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Soil_Black",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Barley",
    "Crop_Cotton",
    "Crop_Ground Nuts",
    "Crop_Maize",
    "Crop_Millets",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Tobacco",
    "Crop_Wheat",
]
eng_train_ohe = create_preprocessor(features_in=ohe_features).fit_transform(train)[ohe_features+['Fertilizer Name']]
display(eng_train)
display(eng_train.dtypes)
display(eng_train_ohe)
display(eng_train_ohe.dtypes)

/home/damian/Desktop/Projects/ml-fertilizers/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_function_transformer.py:345: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(
/home/damian/Desktop/Projects/ml-fertilizers/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_function_transformer.py:345: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


,pca0,pca1,Temparature,Humidity,Moisture,Soil,Crop,Nitrogen,Potassium,Phosphorous,Fertilizer Name,Env_Stress_Index,NPK_Index,Temp_bin,Humidity_bin,Moisture_bin,SoilNutrients,SoilNutrient_Ratio,Temp_Humidity,Temp_Moisture
id,,,,,,,,,,,,,,,,,,,,
0,9.162230,-8.996283,37,70,36,Clayey,Sugarcane,36,4,5,28-28,46.6,20.3,high,very_high,medium,45,3.600000,2590,1332
1,-19.841117,-7.897427,27,69,65,Sandy,Millets,30,6,18,28-28,51.0,21.6,medium,high,very_high,54,1.200000,1863,1755
2,13.176970,-1.994115,29,63,32,Sandy,Millets,24,12,16,17-17-17,40.1,19.2,medium,high,medium,52,0.827586,1827,928
3,-8.816332,-0.942070,35,62,54,Sandy,Barley,39,12,4,10-26-26,48.8,23.1,high,high,high,55,2.294118,2170,1890
4,2.194932,3.026816,35,58,43,Red,Paddy,37,2,16,DAP,44.3,23.7,high,high,high,55,1.947368,2030,1505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,15.157477,-7.992979,25,69,30,Clayey,Maize,8,16,6,28-28,39.7,9.0,medium,high,medium,30,0.347826,1725,750
749996,-12.820711,-2.934131,37,64,58,Loamy,Sugarcane,38,8,20,17-17-17,51.4,26.6,high,high,high,66,1.310345,2368,2146
749997,-13.833277,-6.927904,35,68,59,Sandy,Ground Nuts,6,11,29,10-26-26,52.1,13.9,high,high,high,46,0.146341,2380,2065


pca0                   float64
pca1                   float64
Temparature             uint16
Humidity                uint16
Moisture                uint16
Soil                  category
Crop                  category
Nitrogen                uint16
Potassium               uint16
Phosphorous             uint16
Fertilizer Name         object
Env_Stress_Index       float64
NPK_Index              float64
Temp_bin              category
Humidity_bin          category
Moisture_bin          category
SoilNutrients           uint16
SoilNutrient_Ratio     float64
Temp_Humidity           uint16
Temp_Moisture           uint16
dtype: object

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil_Black,Soil_Clayey,Soil_Loamy,Soil_Red,...,Crop_Ground Nuts,Crop_Maize,Crop_Millets,Crop_Oil seeds,Crop_Paddy,Crop_Pulses,Crop_Sugarcane,Crop_Tobacco,Crop_Wheat,Fertilizer Name
id,,,,,,,,,,,,,,,,,,,,,
0,37,70,36,36,4,5,0,1,0,0,...,0,0,0,0,0,0,1,0,0,28-28
1,27,69,65,30,6,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,28-28
2,29,63,32,24,12,16,0,0,0,0,...,0,0,1,0,0,0,0,0,0,17-17-17
3,35,62,54,39,12,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10-26-26
4,35,58,43,37,2,16,0,0,0,1,...,0,0,0,0,1,0,0,0,0,DAP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,25,69,30,8,16,6,0,1,0,0,...,0,1,0,0,0,0,0,0,0,28-28
749996,37,64,58,38,8,20,0,0,1,0,...,0,0,0,0,0,0,1,0,0,17-17-17
749997,35,68,59,6,11,29,0,0,0,0,...,1,0,0,0,0,0,0,0,0,10-26-26


Temparature                   uint16
Humidity                      uint16
Moisture                      uint16
Nitrogen                      uint16
Potassium                     uint16
Phosphorous                   uint16
Soil_Black          Sparse[uint8, 0]
Soil_Clayey         Sparse[uint8, 0]
Soil_Loamy          Sparse[uint8, 0]
Soil_Red            Sparse[uint8, 0]
Soil_Sandy          Sparse[uint8, 0]
Crop_Barley         Sparse[uint8, 0]
Crop_Cotton         Sparse[uint8, 0]
Crop_Ground Nuts    Sparse[uint8, 0]
Crop_Maize          Sparse[uint8, 0]
Crop_Millets        Sparse[uint8, 0]
Crop_Oil seeds      Sparse[uint8, 0]
Crop_Paddy          Sparse[uint8, 0]
Crop_Pulses         Sparse[uint8, 0]
Crop_Sugarcane      Sparse[uint8, 0]
Crop_Tobacco        Sparse[uint8, 0]
Crop_Wheat          Sparse[uint8, 0]
Fertilizer Name               object
dtype: object

# SEQUENTIAL FEATRURE SELECTION

In [13]:
import numpy as np


def mapk_scorer(estimator, X, y_true, k=3):
    """
    Uses estimator.predict_proba to compute MAP@k.
    y_val contains integer-encoded true labels.
    """
    probas = estimator.predict_proba(X)
    topk   = np.argsort(probas, axis=1)[:, -k:][:, ::-1]  # shape: (n_samples, k)
    scores = []
    for i, true_label in enumerate(y_true):
        preds = topk[i]
        score = 0.0
        hits  = 0
        seen  = set()
        for rank, p in enumerate(preds):
            if p == true_label and p not in seen:
                hits += 1
                score += hits / (rank + 1)
                seen.add(p)
        scores.append(score / 1.0)  # each actual list has length=1
    return np.mean(scores)


In [ ]:
from typing import Literal
from catboost import CatBoostClassifier

# NOTE: Doesn't really work with cat_features
class CatBoostWithCatFeatures(CatBoostClassifier):
    def __init__(self, extended_cat_features: Optional[Union[List, Literal['auto']]] = None, **kwargs):
        super().__init__(**kwargs)
        self._extended_cat_features = extended_cat_features

    def fit(self, X, y, **kwargs): # type: ignore
        cat_auto_detect = self._extended_cat_features == 'auto'
        cat_feats = self._extended_cat_features if isinstance(self._extended_cat_features, list) else None


        # If X is a DataFrame and cat_features are names, convert to indices
        if hasattr(X, 'columns'):
            if cat_feats is not None:
                # Convert names to indices if needed
                cat_feats = [col for col in cat_feats if col in X.columns]
            elif cat_auto_detect:
                # Auto-detect object or category dtype columns
                cat_feats = [X.columns[i] for i, dt in enumerate(X.dtypes) if dt.name in ['object', 'category']]

        elif isinstance(X, np.ndarray):
            if cat_feats is not None:
                # Assume user passed indices
                cat_feats = [col for col in cat_feats if isinstance(col, int)]
            elif cat_auto_detect:
                # Auto-detect non-numeric columns
                # Check if each column is numeric
                cat_feats = []
                for i in range(X.shape[1]):
                    col = X[:, i]
                    # Try converting to float; if fails, it's categorical
                    try:
                        col.astype(float)
                    except (ValueError, TypeError):
                        cat_feats.append(i)
        else:
            # Fallback: don't set cat_features
            cat_feats = None

        return super().fit(X, y, cat_features=cat_feats, **kwargs)

print(feat_dict["cat_features"])

catboost_fast = CatBoostWithCatFeatures(
    iterations=800,
    learning_rate=0.1,
    depth=5,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=42,
    verbose=100,
    allow_writing_files=False,
    thread_count=-1,
    extended_cat_features=feat_dict["cat_features"]
)




In [16]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from ml_fertilizers.lib.logger import setup_logger 

logger = setup_logger(__name__)

class XGBClassifierGPU(XGBClassifier):
    """SHEESH! This is a GPU compatible version of XGBClassifier. It uses the GPU for training and prediction if possible."""

    def __init__(self, **kwargs):

        kwargs.setdefault("device", "cuda")
        kwargs.setdefault("tree_method", "hist")
        self._user_enable_categorical = kwargs.get("enable_categorical", False)

        super().__init__(**kwargs)
 

    def _set_gpu(self, use_gpu: bool) -> "XGBClassifierGPU":
        """Toggle GPU usage for XGBoost."""
        if use_gpu:
            logger.info("Using GPU for XGBoost.")
            self.set_params(
                tree_method="hist", device="cuda", predictor="gpu_predictor"
            )
        else:
            logger.warning("Using CPU for XGBoost. This may be slower than using GPU.")
            self.set_params(tree_method=None, device=None)

        return self
 

    def fit(self, X, y, **kwargs): 
        if isinstance(X, np.ndarray) and self._user_enable_categorical:
            X = pd.DataFrame(X)
            for col in X.columns:
                try:
                    X[col] = X[col].astype(float)
                except ValueError:
                    X[col] = X[col].astype("category")


        X_fin = X
        y_fin = y 
        self_fitted = super().fit(X_fin, y_fin, **kwargs)
        del X_fin
        del y_fin

        # if hasattr(self_fitted, "get_booster"):
        #     self_fitted.get_booster().free_raw_data()

        return self_fitted

    def predict(self, X, **kwargs):
        # X_fin = self._to_gpu_array(X)
        if isinstance(X, np.ndarray) and self._user_enable_categorical:
            X = pd.DataFrame(X)
            for col in X.columns:
                try:
                    X[col] = X[col].astype(float)
                except ValueError:
                    X[col] = X[col].astype("category")
        X_fin = X
        y_pred = super().predict(X_fin, **kwargs)
        del X_fin
        return y_pred

    def predict_proba(self, X, **kwargs):
        # X_fin = self._to_gpu_array(X)
        if isinstance(X, np.ndarray) and self._user_enable_categorical:
            X = pd.DataFrame(X)
            for col in X.columns:
                try:
                    X[col] = X[col].astype(float)
                except ValueError:
                    X[col] = X[col].astype("category")

        X_fin = X
        return super().predict_proba(X_fin, **kwargs)

# xgb_fast = XGBClassifierGPU(
#     n_estimators=600,
#     learning_rate=0.1,
#     max_depth=5,
#     objective='multi:softprob',
#     eval_metric='mlogloss',
#     random_state=69,
#     enable_categorical=True,
# )._set_gpu(True)

logistic_regression = LogisticRegression(
    n_jobs=-1,
)

sfs = SFS(
    logistic_regression, # type: ignore
    k_features=15,
    forward=True,
    scoring=mapk_scorer,
    cv=3,
    verbose=1,
    n_jobs=1,
)

# X_train_sub = eng_train.drop(columns=['Fertilizer Name']).sample(frac=0.2, random_state=69)
X_train_sub = eng_train_ohe.drop(columns=['Fertilizer Name']).sample(frac=0.2, random_state=69)
y_train_sub = eng_train.loc[X_train_sub.index, 'Fertilizer Name']

sfs.fit(X_train_sub, y_train_sub.astype('category').cat.codes)  # type: ignore


Features: 2/15/home/damian/Desktop/Projects/ml-fertilizers/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/damian/Desktop/Projects/ml-fertilizers/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

SequentialFeatureSelector(cv=3, estimator=LogisticRegression(n_jobs=-1),
                          k_features=(15, 15),
                          scoring=<function mapk_scorer at 0x70f53acb7ba0>,
                          verbose=1)

In [19]:
# pkl.dump(sfs, open("sfs_catboost.pkl", "wb"))
from ml_fertilizers.utils import PathManager

# pkl.dump(sfs, open(PathManager.cwd.value/'lr_sfs_final.pkl', "wb"))  # Save the SFS object
# sfs_loaded = pkl.load(open(PathManager.cwd.value/'sfs_catboost_final.pkl', "rb"))  # Load the SFS object if you have it saved
sfs_loaded = sfs
selected_order = []
prev_set = set()
for k in sorted(sfs_loaded.subsets_.keys()):
    cur_set = set(sfs_loaded.subsets_[k]["feature_idx"])
    added   = cur_set - prev_set
    if len(added) == 1:
        feat_idx = added.pop()
        feat_name = X_train_sub.columns[feat_idx]
        avg_score = sfs_loaded.subsets_[k]["avg_score"]
        selected_order.append((k, feat_name, avg_score))
    prev_set = cur_set

# 3.7 Display the selection order and MAP@3
print("Feature | Order → Feature Name         | MAP@3")
print("--------|-------------------------------|--------")
for k, feat, score in selected_order:
    print(f"{k:>2d}      → {feat:<30s}   {score:.5f}")
[feat for (k,feat,score) in selected_order]

Feature | Order → Feature Name         | MAP@3
--------|-------------------------------|--------
 1      → Moisture                         0.28278
 2      → Crop_Oil seeds                   0.28368
 3      → Crop_Wheat                       0.28455
 4      → Crop_Cotton                      0.28543
 5      → Crop_Maize                       0.28522
 6      → Crop_Tobacco                     0.28537
 7      → Humidity                         0.28555
 8      → Potassium                        0.28568
 9      → Crop_Pulses                      0.28529
10      → Soil_Black                       0.28610
11      → Crop_Millets                     0.28610
12      → Nitrogen                         0.28649
13      → Crop_Barley                      0.28641
14      → Soil_Loamy                       0.28670
15      → Crop_Paddy                       0.28664


['Moisture',
 'Crop_Oil seeds',
 'Crop_Wheat',
 'Crop_Cotton',
 'Crop_Maize',
 'Crop_Tobacco',
 'Humidity',
 'Potassium',
 'Crop_Pulses',
 'Soil_Black',
 'Crop_Millets',
 'Nitrogen',
 'Crop_Barley',
 'Soil_Loamy',
 'Crop_Paddy']

# CATBOOST RFE

In [ ]:
from catboost import Pool

X_cat_rfe_train = eng_train.drop(columns=['Fertilizer Name']).sample(frac=0.1, random_state=42)
y_cat_rfe_train = eng_train.loc[X_cat_rfe_train.index, 'Fertilizer Name']

X_cat_rfe_test = eng_train.drop(columns=['Fertilizer Name']).sample(frac=0.1, random_state=43)
y_cat_rfe_test = eng_train.loc[X_cat_rfe_train.index, 'Fertilizer Name']

train_pool = Pool(X_cat_rfe_train, y_cat_rfe_train, cat_features=feat_dict['cat_features'])
test_pool = Pool(X_cat_rfe_test, y_cat_rfe_test, cat_features=feat_dict['cat_features'])


summary = catboost_fast.select_features(
    train_pool,
    eval_set=test_pool,
    num_features_to_select=8,
    features_for_select=X_cat_rfe_train.columns.tolist(),
    steps=5,
    algorithm='RecursiveByShapValues',
    train_final_model=True,
    logging_level='Silent',
    plot=False,
)

summary

In [ ]:
import matplotlib.pyplot as plt

removed_features = summary['loss_graph']['removed_features_count']
loss_values = summary['loss_graph']['loss_values']

plt.figure(figsize=(8, 5))
plt.plot(removed_features, loss_values, marker='o')
plt.title('Feature Selection Loss Curve')
plt.xlabel('Number of Features Removed')
plt.ylabel('Validation Loss')
plt.grid(True)
plt.show()

# RFE

In [ ]:
import json
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

rfe_data = pd.get_dummies(eng_train.drop(columns=['Fertilizer Name']).copy().sample(frac=0.25, random_state=42), drop_first=True)
y = eng_train.loc[rfe_data.index, 'Fertilizer Name'].astype('category').cat.codes  # Convert to integer codes for RFE
 
catboost_rfe = CatBoostClassifier(
    iterations=800,
    learning_rate=0.1,
    depth=5,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=42,
    verbose=0,
    allow_writing_files=False,
    thread_count=-1,
)


xgb_rfe = XGBClassifier(
    n_estimators=800,
    learning_rate=0.1,
    max_depth=5,
    objective='multi:softmax',
    eval_metric='mlogloss',
    random_state=42,
    verbosity=0,
    tree_method='hist',
    n_jobs=-1,
)

lgbm_rfe = LGBMClassifier(
    n_estimators=800,
    learning_rate=0.1,
    max_depth=5,
    objective='multiclass',
    eval_metric='multi_logloss',
    random_state=42,
    verbosity=0,
    n_jobs=-1,
)

rfe_models = [
    catboost_rfe,
    xgb_rfe,
    lgbm_rfe,
]

scoring_features_n = [30, 25, 20, 15, 10, 5]
ans : Dict[str,List[str]] = dict()

for rfe_model in rfe_models:
    base_features = rfe_data.columns.tolist()

    for n_features in sorted(scoring_features_n, reverse=True):
        print(
            f"Training RFE {rfe_model.__class__.__name__} {n_features} features"
        )
        current_features = (
            rfe_data[base_features]
            .columns[
                RFE(rfe_model, n_features_to_select=n_features, step=2) # type: ignore
                .fit(rfe_data[base_features], y)
                .get_support()
            ]
            .tolist()
        )

        original_features = current_features.copy()

        ans[f"{rfe_model.__class__.__name__}_{n_features}"] = current_features
        print(f"Selected features ({len(current_features)}): {current_features}")

        base_features = [col for col in current_features]


print(json.dumps(ans, indent=2))

json.dump(
    ans,
    open(PathManager.cwd.value/'rfe_features.json', "w"),
    indent=2,
    ensure_ascii=False,
)

# FEATURE SETS

In [ ]:
cat_set = [
  'Temparature',
  'Soil',
  'Potassium',
  'Phosphorous',
  'NPK_Index',
  'Temp_bin',
  'Humidity_bin',
  'Moisture_bin'
]

rfe_dict = {
  # "CatBoostClassifier_cat_features": cat_set,
  # "CatBoostClassifier_30": [
  #   "Temparature",
  #   "Humidity",
  #   "Moisture",
  #   "Nitrogen",
  #   "Potassium",
  #   "Phosphorous",
  #   "Env_Stress_Index",
  #   "NPK_Index",
  #   "PCA_Temparature",
  #   "Soil_Clayey",
  #   "Soil_Loamy",
  #   "Soil_Red",
  #   "Soil_Sandy",
  #   "Crop_Cotton",
  #   "Crop_Ground Nuts",
  #   "Crop_Maize",
  #   "Crop_Millets",
  #   "Crop_Oil seeds",
  #   "Crop_Paddy",
  #   "Crop_Pulses",
  #   "Crop_Sugarcane",
  #   "Crop_Tobacco",
  #   "Crop_Wheat",
  #   "Temp_bin_high",
  #   "Temp_bin_very_high",
  #   "Humidity_bin_medium",
  #   "Humidity_bin_high",
  #   "Humidity_bin_very_high",
  #   "Moisture_bin_high",
  #   "Moisture_bin_very_high"
  # ],
  "CatBoostClassifier_25": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Cotton",
    "Crop_Ground Nuts",
    "Crop_Maize",
    "Crop_Millets",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Tobacco",
    "Crop_Wheat",
    "Temp_bin_high",
    "Humidity_bin_high"
  ],
  "CatBoostClassifier_20": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Ground Nuts",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Tobacco",
    "Temp_bin_high"
  ],
  "CatBoostClassifier_15": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane"
  ],
  "CatBoostClassifier_10": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Sandy"
  ],
  # "CatBoostClassifier_5": [
  #   "Humidity",
  #   "Nitrogen",
  #   "Potassium",
  #   "Phosphorous",
  #   "PCA_Temparature"
  # ],
  # "XGBClassifier_30": [
  #   "Temparature",
  #   "Humidity",
  #   "Moisture",
  #   "Nitrogen",
  #   "Potassium",
  #   "Phosphorous",
  #   "Env_Stress_Index",
  #   "NPK_Index",
  #   "PCA_Temparature",
  #   "Soil_Clayey",
  #   "Soil_Loamy",
  #   "Soil_Red",
  #   "Soil_Sandy",
  #   "Crop_Cotton",
  #   "Crop_Ground Nuts",
  #   "Crop_Maize",
  #   "Crop_Millets",
  #   "Crop_Oil seeds",
  #   "Crop_Paddy",
  #   "Crop_Pulses",
  #   "Crop_Sugarcane",
  #   "Crop_Tobacco",
  #   "Crop_Wheat",
  #   "Temp_bin_high",
  #   "Temp_bin_very_high",
  #   "Humidity_bin_medium",
  #   "Humidity_bin_high",
  #   "Humidity_bin_very_high",
  #   "Moisture_bin_medium",
  #   "Moisture_bin_high"
  # ],
  "XGBClassifier_25": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Cotton",
    "Crop_Ground Nuts",
    "Crop_Maize",
    "Crop_Millets",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Tobacco",
    "Crop_Wheat",
    "Temp_bin_high",
    "Humidity_bin_high",
    "Moisture_bin_high"
  ],
  "XGBClassifier_20": [
    "Temparature",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Soil_Clayey",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Cotton",
    "Crop_Ground Nuts",
    "Crop_Maize",
    "Crop_Millets",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Wheat",
    "Temp_bin_high",
    "Humidity_bin_high",
    "Moisture_bin_high"
  ],
  "XGBClassifier_15": [
    "Moisture",
    "Potassium",
    "Phosphorous",
    "Soil_Clayey",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Ground Nuts",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Wheat",
    "Temp_bin_high",
    "Humidity_bin_high",
    "Moisture_bin_high"
  ],
  "XGBClassifier_10": [
    "Moisture",
    "Phosphorous",
    "Soil_Clayey",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Ground Nuts",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Temp_bin_high"
  ],
  # "XGBClassifier_5": [
  #   "Moisture",
  #   "Phosphorous",
  #   "Soil_Sandy",
  #   "Crop_Pulses",
  #   "Crop_Sugarcane"
  # ],
  # "LGBMClassifier_30": [
  #   "Temparature",
  #   "Humidity",
  #   "Moisture",
  #   "Nitrogen",
  #   "Potassium",
  #   "Phosphorous",
  #   "Env_Stress_Index",
  #   "NPK_Index",
  #   "PCA_Temparature",
  #   "Soil_Clayey",
  #   "Soil_Loamy",
  #   "Soil_Red",
  #   "Soil_Sandy",
  #   "Crop_Cotton",
  #   "Crop_Ground Nuts",
  #   "Crop_Maize",
  #   "Crop_Millets",
  #   "Crop_Oil seeds",
  #   "Crop_Paddy",
  #   "Crop_Pulses",
  #   "Crop_Sugarcane",
  #   "Crop_Tobacco",
  #   "Crop_Wheat",
  #   "Temp_bin_medium",
  #   "Temp_bin_high",
  #   "Temp_bin_very_high",
  #   "Humidity_bin_medium",
  #   "Humidity_bin_high",
  #   "Moisture_bin_medium",
  #   "Moisture_bin_high"
  # ],
  "LGBMClassifier_25": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Cotton",
    "Crop_Ground Nuts",
    "Crop_Maize",
    "Crop_Millets",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Sugarcane",
    "Crop_Tobacco",
    "Crop_Wheat",
    "Temp_bin_high",
    "Humidity_bin_high"
  ],
  "LGBMClassifier_20": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Maize",
    "Crop_Millets",
    "Crop_Oil seeds",
    "Crop_Paddy",
    "Crop_Pulses",
    "Crop_Tobacco",
    "Crop_Wheat"
  ],
  "LGBMClassifier_15": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Clayey",
    "Soil_Loamy",
    "Soil_Red",
    "Soil_Sandy",
    "Crop_Oil seeds",
    "Crop_Paddy"
  ],
  "LGBMClassifier_10": [
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous",
    "Env_Stress_Index",
    "NPK_Index",
    "PCA_Temparature",
    "Soil_Loamy"
  ],
  # "LGBMClassifier_5": [
  #   "Nitrogen",
  #   "Phosphorous",
  #   "Env_Stress_Index",
  #   "NPK_Index",
  #   "PCA_Temparature"
  # ],
  "XGBClassifier_SFS_Baseline_Kaggle": ["Moisture", "Phosphorous", "Potassium", "Soil Type_Black", "Nitrogen", "Soil Type_Sandy", "Crop Type_Sugarcane", "Temparature", "Crop Type_Oil seeds", "Crop Type_Cotton"]
}

# TESTING

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
def evaluate(estimator, X, y, cv=3) -> float:

    scores = cross_val_score(estimator, X, y.astype('category').cat.codes, cv=cv, scoring=mapk_scorer)
    return scores.mean()



In [ ]:
ans = {}

X_raw = eng_train.drop(columns=['Fertilizer Name']).sample(frac=0.6, random_state=42)
X_eval = pd.get_dummies(X_raw, drop_first=True)
y_eval = eng_train.loc[X_raw.index, 'Fertilizer Name']


catboost_test = CatBoostClassifier(
    iterations=800,
    learning_rate=0.1,
    depth=5,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=42,
    verbose=0,
    allow_writing_files=False,
    thread_count=-1,
)


xgb_test = XGBClassifier(
    n_estimators=800,
    learning_rate=0.1,
    max_depth=5,
    objective='multi:softmax',
    eval_metric='mlogloss',
    random_state=42,
    verbosity=0,
    tree_method='hist',
    n_jobs=-1,
)

lgbm_test = LGBMClassifier(
    n_estimators=800,
    learning_rate=0.1,
    max_depth=5,
    objective='multiclass',
    eval_metric='multi_logloss',
    random_state=42,
    verbosity=-1,
    n_jobs=-1,
)

test_models = [
    catboost_test,
    xgb_test,
    lgbm_test,
]


for key, features in rfe_dict.items():
    
    model = None

    for test_model in test_models:
        if key.startswith(test_model.__class__.__name__):
            model = test_model
            break

    if model is None:
        print(f"Model for {key} not found!")
        continue

    print(f"Evaluating {key} with features: {features}")
    score = evaluate(
        model,
        X_eval[features],
        y_eval,
        cv=3
    )
    ans[key] = score
    print(f"Score for {key}: {score:.5f}")


for key in ans.keys():
    print(f"{key}: {ans[key]:.5f}")


json.dump(
    ans,
    open(PathManager.cwd.value/'rfe_scores.json', "w"),
    indent=2,
    ensure_ascii=False,
)

# cat_rfe: 0.28679

# TESTING RESULTS

Evaluating CatBoostClassifier_30 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Crop_Wheat', 'Temp_bin_high', 'Temp_bin_very_high', 'Humidity_bin_medium', 'Humidity_bin_high', 'Humidity_bin_very_high', 'Moisture_bin_high', 'Moisture_bin_very_high']
Score for CatBoostClassifier_30: 0.31385
Evaluating CatBoostClassifier_25 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Crop_Wheat', 'Temp_bin_high', 'Humidity_bin_high']
Score for CatBoostClassifier_25: 0.31389
Evaluating CatBoostClassifier_20 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Ground Nuts', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Temp_bin_high']
Score for CatBoostClassifier_20: 0.31273
Evaluating CatBoostClassifier_15 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane']
Score for CatBoostClassifier_15: 0.31018
Evaluating CatBoostClassifier_10 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Sandy']
Score for CatBoostClassifier_10: 0.30740
Evaluating CatBoostClassifier_5 with features: ['Humidity', 'Nitrogen', 'Potassium', 'Phosphorous', 'PCA_Temparature']
Score for CatBoostClassifier_5: 0.30691
Evaluating XGBClassifier_30 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Crop_Wheat', 'Temp_bin_high', 'Temp_bin_very_high', 'Humidity_bin_medium', 'Humidity_bin_high', 'Humidity_bin_very_high', 'Moisture_bin_medium', 'Moisture_bin_high']
Score for XGBClassifier_30: 0.31977
Evaluating XGBClassifier_25 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Crop_Wheat', 'Temp_bin_high', 'Humidity_bin_high', 'Moisture_bin_high']
Score for XGBClassifier_25: 0.32179
Evaluating XGBClassifier_20 with features: ['Temparature', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Soil_Clayey', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Wheat', 'Temp_bin_high', 'Humidity_bin_high', 'Moisture_bin_high']
Score for XGBClassifier_20: 0.32401
Evaluating XGBClassifier_15 with features: ['Moisture', 'Potassium', 'Phosphorous', 'Soil_Clayey', 'Soil_Red', 'Soil_Sandy', 'Crop_Ground Nuts', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Wheat', 'Temp_bin_high', 'Humidity_bin_high', 'Moisture_bin_high']
Score for XGBClassifier_15: 0.31314
Evaluating XGBClassifier_10 with features: ['Moisture', 'Phosphorous', 'Soil_Clayey', 'Soil_Red', 'Soil_Sandy', 'Crop_Ground Nuts', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Temp_bin_high']
Score for XGBClassifier_10: 0.30568
Evaluating XGBClassifier_5 with features: ['Moisture', 'Phosphorous', 'Soil_Sandy', 'Crop_Pulses', 'Crop_Sugarcane']
Score for XGBClassifier_5: 0.30241
Evaluating LGBMClassifier_30 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Crop_Wheat', 'Temp_bin_medium', 'Temp_bin_high', 'Temp_bin_very_high', 'Humidity_bin_medium', 'Humidity_bin_high', 'Moisture_bin_medium', 'Moisture_bin_high']
Score for LGBMClassifier_30: 0.31825
Evaluating LGBMClassifier_25 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Cotton', 'Crop_Ground Nuts', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Sugarcane', 'Crop_Tobacco', 'Crop_Wheat', 'Temp_bin_high', 'Humidity_bin_high']
Score for LGBMClassifier_25: 0.31816
Evaluating LGBMClassifier_20 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Maize', 'Crop_Millets', 'Crop_Oil seeds', 'Crop_Paddy', 'Crop_Pulses', 'Crop_Tobacco', 'Crop_Wheat']
Score for LGBMClassifier_20: 0.31616
Evaluating LGBMClassifier_15 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Clayey', 'Soil_Loamy', 'Soil_Red', 'Soil_Sandy', 'Crop_Oil seeds', 'Crop_Paddy']
Score for LGBMClassifier_15: 0.31306
Evaluating LGBMClassifier_10 with features: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature', 'Soil_Loamy']
Score for LGBMClassifier_10: 0.30871
Evaluating LGBMClassifier_5 with features: ['Nitrogen', 'Phosphorous', 'Env_Stress_Index', 'NPK_Index', 'PCA_Temparature']
Score for LGBMClassifier_5: 0.29873
CatBoostClassifier_30: 0.31385
CatBoostClassifier_25: 0.31389
CatBoostClassifier_20: 0.31273
CatBoostClassifier_15: 0.31018
CatBoostClassifier_10: 0.30740
CatBoostClassifier_5: 0.30691
XGBClassifier_30: 0.31977
XGBClassifier_25: 0.32179
XGBClassifier_20: 0.32401
XGBClassifier_15: 0.31314
XGBClassifier_10: 0.30568
XGBClassifier_5: 0.30241
LGBMClassifier_30: 0.31825
LGBMClassifier_25: 0.31816
LGBMClassifier_20: 0.31616
LGBMClassifier_15: 0.31306
LGBMClassifier_10: 0.30871
LGBMClassifier_5: 0.29873

# ENSEMBLES

In [ ]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

X_raw = eng_train.drop(columns=['Fertilizer Name']).sample(frac=0.4, random_state=42)
X_eval = pd.get_dummies(X_raw, drop_first=True)
y_eval = eng_train.loc[X_raw.index, 'Fertilizer Name']


catboost_stack = CatBoostClassifier(
    iterations=800,
    learning_rate=0.1,
    depth=5,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=42,
    verbose=0,
    allow_writing_files=False,
    thread_count=-1,
)


xgb_stack = XGBClassifier(
    n_estimators=800,
    learning_rate=0.1,
    max_depth=5,
    objective='multi:softmax',
    eval_metric='mlogloss',
    random_state=42,
    verbosity=0,
    tree_method='hist',
    n_jobs=-1,
)

lgbm_stack = LGBMClassifier(
    n_estimators=800,
    learning_rate=0.1,
    max_depth=5,
    objective='multiclass',
    eval_metric='multi_logloss',
    random_state=42,
    verbosity=-1,
    n_jobs=-1,
)

random_forest_stack = RandomForestClassifier(
    n_estimators=800,
    max_depth=5,
    random_state=42,
    n_jobs=-1,
)

stacking_model = StackingClassifier(
    estimators=[
        ('catboost', catboost_stack),
        ('xgboost', xgb_stack),
        ('lgbm', lgbm_stack),
    ],
    final_estimator=random_forest_stack,
    cv=3,
    n_jobs=-1,
    passthrough=True,
)

stacking_score = evaluate(
    stacking_model,
    X_eval[rfe_dict['XGBClassifier_20']],
    y_eval,
    cv=3
)
print(f"Stacking model score: {stacking_score:.5f}")



# HYPEROPT

In [ ]:
import os
from typing import List
import multiprocessing as mp

import pandas as pd
from catboost import CatBoostClassifier 
from lightgbm import LGBMClassifier
import optuna
from sklearn import clone

from ml_fertilizers.lib.features.FeatureCombination import FeatureCombination
from ml_fertilizers.lib.models.HyperOptCombination import HyperOptCombination
from ml_fertilizers.lib.models.XgbGPU import XGBClassifierGPU
from ml_fertilizers.lib.optymization.TrialParamWrapper import TrialParamWrapper
from ml_fertilizers.lib.optymization.optimization_study import OBJECTIVE_RETURN_TYPE
from ml_fertilizers.lib.utils.garbage_collector import garbage_manager


processes = None
my_combinations: List[HyperOptCombination] = []
job_count = processes if processes is not None else mp.cpu_count()
os.environ["OMP_NUM_THREADS"] = str(job_count)
os.environ["MKL_NUM_THREADS"] = str(job_count)
RANDOM_STATE = 42
gpu=True
models = [
    CatBoostClassifier(thread_count=job_count, random_state=RANDOM_STATE, verbose=False, allow_writing_files=False, loss_function='MultiClass', eval_metric='MultiClass'),
    LGBMClassifier(n_jobs=job_count, verbosity=-1, random_state=RANDOM_STATE, objective='multiclass', eval_metric='multi_logloss'),  # type: ignore
    XGBClassifierGPU(random_state=RANDOM_STATE, n_jobs=job_count, verbosity=0, objective='multi:softmax', eval_metric='mlogloss')._set_gpu(use_gpu=gpu),
]

for key, features in rfe_dict.items():
    for model in models:
        if model.__class__.__name__.startswith(key.split('_')[0]):
            my_combinations.append(
                HyperOptCombination(
                    name=key,
                    model=model,
                    feature_combination=FeatureCombination(
                        name="_".join(key.split('_')[1:]),
                        features=features
                    )
                )
            )



display(my_combinations)

def create_objective(data:pd.DataFrame, model_combination:HyperOptCombination):

    X = pd.get_dummies(data.drop(columns=['Fertilizer Name']), drop_first=False).copy()
    y = data['Fertilizer Name'].astype('category').cat.codes  # Convert to integer codes for RFE
    model = model_combination.model
    model_name = model_combination.name

    def objective(trail: optuna.Trial) -> OBJECTIVE_RETURN_TYPE:
        params = TrialParamWrapper().get_params(
            model_name=model_name,
            trial=trail,
        )        

        pipeline = clone(model).set_params(**params)

        try:

            score = evaluate(pipeline, X[model_combination.feature_combination.features], y, cv=3)

            return score
        
        except optuna.exceptions.TrialPruned as e:
            print(f"Trial {trail.number} was pruned: {e}")
            raise e
        except Exception as e:
            print(f"Error during evaluation of trial {trail.number}: {e}")
            raise e
        finally: 
            garbage_manager.clean()


    return objective


In [ ]:
from ml_fertilizers.lib.optymization.hyper_setup import setup_hyper
from ml_fertilizers.lib.optymization.parrarel_optimization import (
    HyperFunctionDto,
    HyperSetupDto,
)
from ml_fertilizers.utils import PrefixManager, PathManager

model_run = "intial_run"

setup_dto = HyperSetupDto(
    n_optimization_trials=60,
    optimization_timeout=None,
    n_patience=20,
    min_percentage_improvement=0.007,
    model_run=model_run,
    limit_data_percentage=0.75,
    processes=processes,
    max_concurrent_jobs=None,
    output_dir_path=PathManager.output.value,
    hyper_opt_prefix=PrefixManager.hyper.value, 
    study_prefix=PrefixManager.study.value,
    data=eng_train,
    combinations=my_combinations,
    hyper_direction="maximize",
    metadata={},
    force_all_sequential=False,
    omit_names=None,
)


function_dto = HyperFunctionDto(
    create_objective_func=create_objective,
    evaluate_hyperopted_model_func=None,
)

setup_hyper(
    setup_dto=setup_dto,
    function_dto=function_dto
)
